## tranfer learning (전이 학습)
## transferlearning_001.ipynb 

#### 최소한의 노력으로 최대의 효과를 얻고자 하는 것이 목표
#### pypi.org에서 수많은 라이브러리가 있는 것처럼,
#### 분명히 다른 사람이 내가 하고자 하는 작업에 적합한 모델을 만드는데 시간을 보냈을꺼야!
#### 누군가가 딥러닝을 위해 모델을 이미 만들어서 존재한다.


[tensorflow hub](https://tfhub.dev/) : 기존 모델 구성 요소를 위한 저장소

#### 전이 학습을 사용하면, 원래 데이터의 10%만 사용하여 지금까지 최고의 모델이 얻은 것과 동일한 결과를 얻을 수 있다.

#### 우리가 지난주에 한 것처럼 1개의 클래스에 750개 정도의 이미지가 있어도 좋으련만... 
#### 이미지는 구하기도 어렵고, 구하는 시간도 오래 걸립니다. 이미지를 구할 때 귀찮으면 bias가 생길 수도 있다. (내가 좋은 결과를 얻고자 하는 이미지만 보려고 한다)
#### 시간은 없고, 돈도 없고, 이미지는 75개 정도만 있는데 결과를 빨리 얻고 싶은데...

In [1]:
!wget -q https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

In [2]:
import zipfile

zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

# 10_food_classes_10_percent 라는 폴더가 생성됩니다.

In [3]:
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
    print(f"{dirpath} 폴더에는 {len(dirnames)}개의 폴더와 {len(filenames)}개의 파일이 존재합니다.")

# 학습은 지난번보다 10%의 데이터로 하고, 테스트는 원래의 양과 같이 한다.

10_food_classes_10_percent 폴더에는 2개의 폴더와 0개의 파일이 존재합니다.
10_food_classes_10_percent/test 폴더에는 10개의 폴더와 0개의 파일이 존재합니다.
10_food_classes_10_percent/test/ice_cream 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/chicken_curry 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/steak 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/sushi 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/chicken_wings 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/grilled_salmon 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/hamburger 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/pizza 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/ramen 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/fried_rice 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/train 폴더에는 10개의 폴더와 0개의 파일이 존재합니다.
10_food_classes_10_percent/train/ice_cream 폴더에는 0개의 폴더와 75개의 파일이 존재합니다.
10_food_classes_10_percent/train

In [4]:
# 데이터가 준비가 된 상태입니다.
# 데이터를 분석할 준비하기

from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

# data normalization
train_datagen = ImageDataGenerator(rescale = 1/255.) # 값을 0 ~ 1사이로 조정
test_datagen = ImageDataGenerator(rescale = 1/255.)

print("학습 이미지 : ")
train_data_10_percent = train_datagen.flow_from_directory(
    train_dir,
    target_size = IMAGE_SHAPE,
    batch_size = BATCH_SIZE,
    class_mode = "categorical"
)

print("테스트 이미지 : ")
test_data_10_percent = test_datagen.flow_from_directory(
    test_dir,
    target_size = IMAGE_SHAPE,
    batch_size = BATCH_SIZE,
    class_mode = "categorical"
)



학습 이미지 : 
Found 750 images belonging to 10 classes.
테스트 이미지 : 
Found 2500 images belonging to 10 classes.


### callback 설정 (모델이 학습하는 동안 실행하는 것)

#### 학습 중에 또는 학습 후에 수행할 모델에 기능을 더 추가해 주는 것!

#### TensorBoard를 사용!
#### 실험을 추적하고 여러 모델의 성능을 기록한 다음 tensorboard에서 시각적 방식으로
모델을 비교할 수 있습니다. 같은 데이터로 여러 모델의 결과를 비교하는데 유용합니다.

#### 모델 체크포인트 (Model Checkpointing) : 상황에 따라 학습을 중지하고 다시 돌아와서 계속 진행할 수 있도록 모델을 저장

#### 조기 중지 (Early Stopping) : 임의의 시간동안 모델 학습을 진행시키다가 모델이 개선된 것으로 판단이 되면 학습을 자동으로 중단. 대용량 데이터셋이 있고 학습에 얼마나 오래 걸리지 모를 때 유용한 개념

In [5]:
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir
    )
    print(f"TensorBoard 로그 파일을 저장한 디렉토리 : {log_dir}")
    return tensorboard_callback

In [6]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
!pip install tensorflow_hub
!pip freeze > requirements.txt

In [8]:
import tensorflow_hub as hub
from tensorflow.keras import layers

In [9]:
# Resnet 50 V2
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# EfficientNet0
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [10]:
# tensorflow hub URL을 가지고 와서 Keras sequential model을 생성

# model_url : tensorflow hub에 존재하는 모델의 링크
# num_classes : 출력층에서 출력 뉴런의 갯수, 대상 클래스의 수와 같아야 합니다. 기본 설정값을 10을 함

# 아래 함수을 결과는 컴파일 되지 않은 keras sequential model!

def create_model(model_url, num_classes = 10):
    feature_extractor_layer = hub.KerasLayer(
        model_url,
        trainable = False,   # 기본 패턴을 고정
        name = "feature_extraction_layer",
        input_shape = IMAGE_SHAPE + (3, )
    )

    model = tf.keras.Sequential([
        feature_extractor_layer,
        layers.Dense(
            num_classes, 
            activation = "softmax", 
            name = "output_layer"
        )
    ])

    return model

In [11]:
print(train_data_10_percent.num_classes)

10


In [12]:
# 모델 생성
resnet_model = create_model(
    resnet_url,
    num_classes = train_data_10_percent.num_classes
)

# 컴파일
resnet_model.compile(
    loss = "categorical_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [13]:
resnet_history = resnet_model.fit(
    train_data_10_percent,
    epochs = 5,
    steps_per_epoch = len(train_data_10_percent),
    validation_data = test_data_10_percent,
    validation_steps = len(test_data_10_percent),
    callbacks = [
        create_tensorboard_callback(
            dir_name = "tensorflow_hub",
            experiment_name = "resnet50V2"
        )
    ]
)

TensorBoard 로그 파일을 저장한 디렉토리 : tensorflow_hub/resnet50V2/20210619-105356
Epoch 1/5
24/24 [==============================] - 287s 12s/step - loss: 1.9194 - accuracy: 0.3413 - val_loss: 1.2304 - val_accuracy: 0.6108
Epoch 2/5
24/24 [==============================] - 457s 20s/step - loss: 0.9461 - accuracy: 0.7173 - val_loss: 0.9195 - val_accuracy: 0.6972
Epoch 3/5
24/24 [==============================] - 441s 19s/step - loss: 0.6344 - accuracy: 0.8307 - val_loss: 0.7768 - val_accuracy: 0.7452
Epoch 4/5
24/24 [==============================] - 501s 22s/step - loss: 0.4964 - accuracy: 0.8733 - val_loss: 0.7268 - val_accuracy: 0.7624
Epoch 5/5
24/24 [==============================] - 417s 18s/step - loss: 0.3981 - accuracy: 0.9013 - val_loss: 0.6966 - val_accuracy: 0.7712
